## Forecasting a Time Series

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

### Creating our own Time Series data

##### When dealing with time series (and other types of sequences such as sentences), the input features are generally represented as 3D arrays of shape [batch size, time steps, dimensionality], where dimensionality is 1 for univariate time series and more for multivariate time series.

In [2]:
def generate_time_series(batch_size, n_steps):
    """
    This function creates as many time series as requested (via the batch_size argument), 
    each of length n_steps, and there is just one value per time step in
    each series (i.e., all series are univariate). 
    The function returns a NumPy array of shape [batch size, time steps, 1], 
    where each series is the sum of two sine waves of fixed amplitudes but random frequencies and phases, 
    plus a bit of noise.
    """
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1 
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2 
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise 
    return series[..., np.newaxis].astype(np.float32)

#### Creating train/test/valid sets

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

In [4]:
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1] 
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [5]:
X_train, X_train.shape

(array([[[ 0.03852015],
         [ 0.12723805],
         [ 0.2254296 ],
         ...,
         [ 0.08864839],
         [ 0.23562351],
         [ 0.42870605]],
 
        [[ 0.41015875],
         [ 0.5966952 ],
         [ 0.5837619 ],
         ...,
         [ 0.49573314],
         [ 0.40426284],
         [ 0.38438833]],
 
        [[ 0.12796116],
         [-0.02429285],
         [-0.2951309 ],
         ...,
         [ 0.27147624],
         [ 0.30975848],
         [ 0.230126  ]],
 
        ...,
 
        [[-0.1812984 ],
         [-0.29633474],
         [-0.40058154],
         ...,
         [ 0.62124664],
         [ 0.39424926],
         [ 0.09833404]],
 
        [[-0.29095381],
         [-0.47718957],
         [-0.56808585],
         ...,
         [ 0.28008774],
         [ 0.19765839],
         [ 0.18636045]],
 
        [[ 0.52851313],
         [ 0.50694364],
         [ 0.36168638],
         ...,
         [ 0.18446174],
         [ 0.31761727],
         [ 0.3488113 ]]], dtype=float32), (700

### Fully connected network. 

Since it expects a flat list of features for each input, we need to add a Flatten layer. Let’s just use
a simple Linear Regression model so that each prediction will be a linear combination of the values in the time series:


In [25]:
model_funny = keras.models.Sequential([ 
    keras.layers.Flatten(input_shape=[50, 1]), 
    keras.layers.Dense(1)
])

model_funny.compile(loss='mse', optimizer="adam", metrics=["accuracy"])

In [26]:
model_funny.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/10
7000/7000 [==============================] - 0s 60us/sample - loss: 0.1005 - accuracy: 0.0000e+00 - val_loss: 0.0448 - val_accuracy: 0.0000e+00
Epoch 2/10
7000/7000 [==============================] - 0s 28us/sample - loss: 0.0301 - accuracy: 0.0000e+00 - val_loss: 0.0217 - val_accuracy: 0.0000e+00
Epoch 3/10
7000/7000 [==============================] - 0s 28us/sample - loss: 0.0166 - accuracy: 0.0000e+00 - val_loss: 0.0133 - val_accuracy: 0.0000e+00
Epoch 4/10
7000/7000 [==============================] - 0s 28us/sample - loss: 0.0120 - accuracy: 0.0000e+00 - val_loss: 0.0104 - val_accuracy: 0.0000e+00
Epoch 5/10
7000/7000 [==============================] - 0s 30us/sample - loss: 0.0101 - accuracy: 0.0000e+00 - val_loss: 0.0091 - val_accuracy: 0.0000e+00
Epoch 6/10
7000/7000 [==============================] - 0s 28us/sample - loss: 0.0091 - accuracy: 0.0000e+00 - val_loss: 0.0082 - val_accuracy: 0.0000e+00
Epoch 7/10
7000/7000 [

In [27]:
model_funny.evaluate(X_valid, y_valid)

2000/2000 [==============================] - 0s 20us/sample - loss: 0.0058 - accuracy: 0.0000e+00


[0.005755842175334692, 0.0]

### Implementing a single RNN

In [28]:
model = keras.models.Sequential([ 
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

model.compile(loss="mse", optimizer='adam')
history = model.fit(X_train, y_train, epochs=20,validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 2s 313us/sample - loss: 0.1615 - val_loss: 0.1254
Epoch 2/20
7000/7000 [==============================] - 2s 232us/sample - loss: 0.1107 - val_loss: 0.0952
Epoch 3/20
7000/7000 [==============================] - 2s 231us/sample - loss: 0.0830 - val_loss: 0.0719
Epoch 4/20
7000/7000 [==============================] - 2s 232us/sample - loss: 0.0641 - val_loss: 0.0572
Epoch 5/20
7000/7000 [==============================] - 2s 233us/sample - loss: 0.0521 - val_loss: 0.0475
Epoch 6/20
7000/7000 [==============================] - 2s 232us/sample - loss: 0.0439 - val_loss: 0.0406
Epoch 7/20
7000/7000 [==============================] - 2s 236us/sample - loss: 0.0380 - val_loss: 0.0355
Epoch 8/20
7000/7000 [==============================] - 2s 245us/sample - loss: 0.0334 - val_loss: 0.0315
Epoch 9/20
7000/7000 [==============================] - 2s 235us/sample - loss: 0.0299 - val_loss: 0.028

By default, the SimpleRNN layer uses the hyperbolic tangent activation function. It works exactly as we saw earlier: the initial state h(init) is set to 0, and it is passed to a single recurrent neuron, along with the value of the first time step, x(0). The neuron computes a weighted sum of these values and applies the hyperbolic tangent activation function to the result, and this gives the first output, y0. In a simple RNN, this output is also the new state h0. This new state is passed to the same recurrent neuron along with the next input value, x(1), and the process is repeated until the last time step. Then the layer just outputs the last value, y49. All of this is performed simultaneously for every time series.

In [29]:
model.evaluate(X_valid, y_valid)

2000/2000 [==============================] - 0s 71us/sample - loss: 0.0134


0.013431698046624661

### Deep RNN

##### By default, recurrent layers in Keras only return the final output. To make them return one output per time step, you must set return_sequences=True, as we will see.

In [30]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 6s 899us/sample - loss: 0.0271 - val_loss: 0.0049
Epoch 2/20
7000/7000 [==============================] - 5s 700us/sample - loss: 0.0043 - val_loss: 0.0037
Epoch 3/20
7000/7000 [==============================] - 5s 760us/sample - loss: 0.0036 - val_loss: 0.0034
Epoch 4/20
7000/7000 [==============================] - 5s 697us/sample - loss: 0.0034 - val_loss: 0.0033
Epoch 5/20
7000/7000 [==============================] - 5s 701us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 6/20
7000/7000 [==============================] - 5s 676us/sample - loss: 0.0032 - val_loss: 0.0032
Epoch 7/20
7000/7000 [==============================] - 5s 671us/sample - loss: 0.0031 - val_loss: 0.0030
Epoch 8/20
7000/7000 [==============================] - 5s 669us/sample - loss: 0.0031 - val_loss: 0.0030
Epoch 9/20
7000/7000 [==============================] - 5s 668us/sample - loss: 0.0030 - val_loss: 0.003

### WARNING!
Make sure to set return_sequences=True for all recurrent layers (except the last one, if you only care about the last output). If you don’t, they will output a 2D array (containing only the output of the last time step) instead of a 3D array (containing outputs for all time steps), and the next recurrent layer will complain that you are not feeding it sequences in the expected 3D format.

In [31]:
model.evaluate(X_valid, y_valid)

2000/2000 [==============================] - 0s 171us/sample - loss: 0.0028


0.0027888535149395467

SimpleRNN layer uses the tanh activation function by default, the predicted values must lie within the range –1 to 1. But what if you want to use another activation function? For both these reasons, it might be preferable to replace the output layer with a Dense layer: it would run slightly faster, the accuracy would be roughly the same, and it would allow us to choose any output activation function we want. If you make this change, also make sure to remove return_sequences=True from the second (now last) recurrent layer:


In [32]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]), 
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

model.evaluate(X_valid, y_valid)

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 645us/sample - loss: 0.0209 - val_loss: 0.0049
Epoch 2/20
7000/7000 [==============================] - 4s 505us/sample - loss: 0.0042 - val_loss: 0.0039
Epoch 3/20
7000/7000 [==============================] - 3s 480us/sample - loss: 0.0035 - val_loss: 0.0033
Epoch 4/20
7000/7000 [==============================] - 3s 476us/sample - loss: 0.0033 - val_loss: 0.0035
Epoch 5/20
7000/7000 [==============================] - 3s 477us/sample - loss: 0.0033 - val_loss: 0.0033
Epoch 6/20
7000/7000 [==============================] - 3s 500us/sample - loss: 0.0031 - val_loss: 0.0029
Epoch 7/20
7000/7000 [==============================] - 3s 487us/sample - loss: 0.0031 - val_loss: 0.0031
Epoch 8/20
7000/7000 [==============================] - 4s 511us/sample - loss: 0.0031 - val_loss: 0.0032
Epoch 9/20
7000/7000 [==============================] - 3s 477us/sample - loss: 0.0030 - val_loss: 0.003

0.002951365625485778